<a href="https://colab.research.google.com/github/dukelin95/baxter_rl/blob/duke_devel/langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -qU langchain
!pip install transformers
!pip install sentencepiece
!pip install accelerate

!pip install pypdf
!pip install chromadb
!pip install sentence_transformers
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.6/975.6 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

from transformers import AutoTokenizer, pipeline, T5ForConditionalGeneration


In [8]:
from flanpipeline import TextGenerationFlanPipeline

In [9]:
# model_id = "gpt2"
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(model_id)
# pipe = pipeline(
#     "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=50
# )
# hf = HuggingFacePipeline(pipeline=pipe)

model_id = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = T5ForConditionalGeneration.from_pretrained(model_id)
pipe = TextGenerationFlanPipeline(model, tokenizer, framework='pt', task='text-generation')

hf = HuggingFacePipeline(pipeline=pipe)

In [10]:
template = """Question: {question}

            Answer:"""

prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=hf, verbose=True)

In [11]:
question = "Give me a haiku"

In [12]:
print(llm_chain.run(question))



> Entering new LLMChain chain...
Prompt after formatting:
Question: Give me a haiku

            Answer:


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(



> Finished chain.
a symphony of birds


In [13]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings 
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo
)

In [14]:
loader = PyPDFLoader('/content/lora.pdf')
pages = loader.load_and_split()
pages.extend(PyPDFLoader('/content/ptuning.pdf').load_and_split())
pages.extend(PyPDFLoader('/content/prompttuning.pdf').load_and_split())
pages.extend(PyPDFLoader('/content/ptuningv2.pdf').load_and_split())

hf_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
faiss_index = FAISS.from_documents(pages, hf_embeddings)


In [17]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=hf, 
                                 chain_type="stuff", 
                                 retriever=faiss_index.as_retriever())

In [18]:
print(qa.run("What is parameter efficient fine tuning?"))

Token indices sequence length is longer than the specified maximum sequence length for this model (4029 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Prompt Tuning


In [51]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
import os
from getpass import getpass

os.environ['HUGGINGFACEHUB_API_TOKEN'] = getpass()

# initialize HF LLM
flan_t5 = HuggingFaceHub(
    repo_id="google/flan-t5-xl",
    model_kwargs={"temperature":1e-10}
)



green bay packers


In [56]:
# build prompt template for simple question-answering
template = """Question: {question}

Answer: """
prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(
    prompt=prompt,
    llm=flan_t5,
    verbose = True
)

question = "Which NFL team won the Super Bowl in the 2010 season?"

print(llm_chain.run(question))